# Explorando os dados

## Imports necessários

In [1]:
import gzip  # descomprimir arquivos .gz
import pandas as pd
import requests # para baixar a base/amostra para sua máquina
from tqdm.notebook import tqdm # para mostrar barra de progresso


In [3]:
# Habilitar barra de progresso
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


## Baixando o dataset (opcional, caso já o tenha)

In [2]:
# Escrever o arquivo em disco, mostrando barra de progresso
# https://stackoverflow.com/a/37573701

# url = "https://projeto-puc-ai-ml-2022.s3.amazonaws.com/20kSongs.pickle.gz"
# Descomente a linha abaixo e comente a linha acima, caso sua máquina não tenha memória
url = "https://projeto-puc-ai-ml-2022.s3.amazonaws.com/10kSongs_1.pickle.gz"

downloaded_filename = url.split("/")[-1]

response = requests.get(url, stream=True)

# Levantar HTTPError pra qualquer resposta http não esperada
response.raise_for_status()

# Escrever e mostrar progresso
total_size_in_bytes= int(response.headers.get('content-length', 0))
block_size = 1024 # 1 Kb
progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
output_filename = f"../datasets/{downloaded_filename}"
with open(output_filename, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()
if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
    print("ERRO: algo deu errado!")

  0%|          | 0.00/816M [00:00<?, ?iB/s]

### Descomprimindo o arquivo baixado (opcional, caso já o tenha)

In [4]:
import shutil
with gzip.open(output_filename, 'rb') as f_in:
    decompressed_filename = output_filename.replace(".gz", "")
    with open(decompressed_filename, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

## Carregando dataset na memória

Antes de rodar a próxima célula, certifique-se de descomprimir o arquivo do dataset antes. Ex:

`gzip -d 10kSongs_1.pickle.gz`

Alternativa: decomprimir ao mesmo tempo em que carrega na memória

`df = pd.read_pickle("../datasets/10kSongs_1.pickle.gz", compress={"method": "gzip", "compresslevel": 9})`

In [5]:
df = pd.read_pickle(decompressed_filename)
# df = pd.read_pickle("../datasets/10kSongs_1.pickle.gz", compress={"method": "gzip", "compresslevel": 9})


## Explorando

In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   analysis_sample_rate        10000 non-null  int32  
 1   artist_7digitalid           10000 non-null  int32  
 2   artist_familiarity          10000 non-null  float64
 3   artist_hotttnesss           10000 non-null  float64
 4   artist_id                   10000 non-null  object 
 5   artist_latitude             10000 non-null  float64
 6   artist_location             10000 non-null  object 
 7   artist_longitude            10000 non-null  float64
 8   artist_mbid                 10000 non-null  object 
 9   artist_mbtags               10000 non-null  object 
 10  artist_mbtags_count         10000 non-null  object 
 11  artist_name                 10000 non-null  object 
 12  artist_playmeid             10000 non-null  int32  
 13  artist_terms                1000

In [13]:
df.describe()

,analysis_sample_rate,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_latitude,artist_longitude,artist_playmeid,danceability,duration,end_of_fade_in,...,mode,mode_confidence,release_7digitalid,song_hotttnesss,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_7digitalid,year
count,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000
mean,22050.0,58996.100800,0.657510,0.451753,41.681794,-45.486389,24945.641100,0.0,245.463986,0.898147,...,0.670300,0.480704,316255.683100,0.464064,235.715078,125.154325,3.593300,0.501146,3.514958e+06,1996.847700
std,0.0,92707.901847,0.103580,0.081716,16.207656,56.611602,43464.204468,0.0,113.451018,2.069717,...,0.470128,0.191856,234072.533995,0.207677,110.661045,35.244338,1.233145,0.373282,2.604464e+06,11.067916
min,22050.0,-1.000000,0.000000,0.000000,-45.874500,-157.857620,-1.000000,0.0,2.298320,0.000000,...,0.000000,0.000000,39.000000,0.000000,2.298000,0.000000,0.000000,0.000000,4.430000e+02,1951.000000
25%,22050.0,5571.000000,0.595042,0.397626,37.167930,-87.632410,-1.000000,0.0,181.877097,0.000000,...,0.000000,0.366000,109861.000000,0.345802,173.596500,99.425500,3.000000,0.091000,1.142070e+06,1992.000000
50%,22050.0,18914.000000,0.648044,0.443030,42.312560,-73.666580,2635.000000,0.0,229.589750,0.208500,...,1.000000,0.492000,285079.500000,0.494490,219.843500,122.114000,4.000000,0.534000,3.206906e+06,2000.000000
75%,22050.0,64232.000000,0.720908,0.495863,51.506320,-0.127140,30909.000000,0.0,285.171792,0.459250,...,1.000000,0.608000,511260.500000,0.609281,273.876750,146.019000,4.000000,0.855250,5.674424e+06,2005.000000
max,22050.0,603509.000000,0.948290,0.969246,69.651020,175.471310,242965.000000,0.0,2488.868120,39.660000,...,1.000000,1.000000,823409.000000,0.997604,2485.899000,252.943000,7.000000,1.000000,9.088514e+06,2010.000000
